TRANSFER LAERNING

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

def preprocess_image(file_path):
  #read the image file
  img = tf.io.read_file(file_path)
  #decode the image to a tensor
  img = tf.image.decode_image(img, channels=3, expand_animations=False)
  #resize the image to the desired size
  img = tf.image.resize(img, IMG_SIZE)
  #normalize the image
  img = tf.cast(img, tf.float32)/255.0
  return img

def create_dataset(directory):
  image_paths = []
  labels = []
  class_names = sorted(os.listdir(directory))
  for class_name in class_names:
    class_dir = os.path.join(directory, class_name)
    for img_name in os.listdir(class_dir):
      img_path = os.path.join(class_dir, img_name)
      image_paths.append(img_path)
      labels.append(class_names.index(class_name))

  def load_and_preprocess_image(path, label):
    return preprocess_image(path), label

  dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
  dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
  dataset = dataset.batch(BATCH_SIZE)
  dataset = dataset.prefetch(tf.data.AUTOTUNE)
  return dataset

#pahs to your data directories
train_dir = 'C:/Users/admin/Downloads/archive/train'
validation_dir = 'C:/Users/admin/Downloads/archive/validation'

#create datasets
train_dataset = create_dataset(train_dir)
validation_dataset = create_dataset(validation_dir)

#print dataset info
print("Number of training batches:", len(train_dataset))
print("Number of validation batches:", len(validation_dataset))

#function to visualize some images (for debgguging)
def visualize_dataset(dataset, num_images=5):
  import matplotlib.pyplot as plt
  plt.figure(figsize=(15, 3))
  for images, labels in dataset.take(1):
    for i in range(min(num_images, len(images))):
      ax = plt.subplot(1, num_images, i + 1)
      plt.imshow(images[i])
      plt.title('Cat' if labels[i] == 0 else 'Dog')
      plt.axis('off')
  plt.show()

#visualize some images from the training dataset
visualize_dataset(train_dataset)
visualize_dataset(validation_dataset)
